In [2]:
import os
import sys
import pandas
import numpy, scipy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [7]:
file_name = "/Users/simondi/PROJECTS/target_infect_x_project/src/tix-analysis/data/single_cell_imaging_data.tsv"

In [5]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [8]:
df = spark.read.csv(file_name, sep="\t", header=True)

In [9]:
for i, x in enumerate(df.columns):
    if x.startswith("cells") or x.startswith("is_infected"):
        df = df.withColumn(x, df[x].cast("double"))
df = df.fillna(0)

In [10]:
cols = [x for x in df.columns if x.startswith("cell")]
assembler = VectorAssembler(
          inputCols=cols,
          outputCol="features")
df = assembler.transform(df)

In [3]:
from pyspark.ml.classification import GBTClassifier

In [12]:
gbt = GBTClassifier(labelCol="is_infected", maxIter=10)

In [14]:
git = gbt.fit(df)

In [ ]:
git.

In [15]:
df2 = git.transform(df)

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [19]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="is_infected")
accuracy = evaluator.evaluate(df2)

In [20]:
accuracy

1.0

In [21]:
spark.stop()